In [8]:
from itertools import combinations
from pyspark import SparkContext
from collections import defaultdict

In [9]:
'''
case = int(sys.argv[1])
support = sys.argv[2]
input_file_path = sys.argv[3]
output_file_path = sys.argv[4]
'''
case = 1
threshold = 10
input_file_path = "/Users/gopi/Desktop/Assignment2/small2.csv"
output_file_path = "/Users/gopi/Desktop/Assignment2/output1.csv"


In [10]:
sc = SparkContext.getOrCreate()


In [11]:
def apriori(partition, total_length):
    baskets = list(partition)
    #print(baskets)
    unique_items = set()
    for basket in baskets:
        for item in basket:
            unique_items.add(item)
    single_item_sets = list(unique_items)#list(set(i for sub in baskets for i in sub))
    scaled_threshold = (len(baskets) / total_length) * threshold
    #print(scaled_threshold)
    candidates = list(combinations(single_item_sets,1))
    support = get_support_candidates(candidates, baskets)
    pruned_candidates = prune_candidates(support, scaled_threshold)
    #print(pruned_candidates)
    
    result_set = []
    result_set.append(pruned_candidates)
    count = 2
    while(len(pruned_candidates) > 0) :
        item_sets = combinations(pruned_candidates, 2)
        item_sets = reformat_item_sets(item_sets, count)
        #print(item_sets)
        support = get_support_candidates(item_sets, baskets)
        pruned_candidates = prune_candidates(support, scaled_threshold)
        if len(pruned_candidates) > 0:
            result_set.append(pruned_candidates)
        count+=1
    #print(result_set)
    return result_set   
    
def reformat_item_sets(item_sets, size):
    result_dict = {}
    for item in item_sets:
        new_tuple = tuple(set(item[0]).union(set(item[1])))
        if(size == len(new_tuple)):
            new_tuple = tuple(sorted(new_tuple))
            if new_tuple not in result_dict:
                result_dict[new_tuple] = None
    return list(result_dict.keys())

def get_single_item_sets(baskets) :
    result = set()
    for basket in baskets:
        for item in basket:
            result.add(item)
    return result

def get_support_candidates(candidates, baskets) :
    support = {}
    for candidate in candidates:
        candidate_set = set(candidate)
        #print(candidate_set)
        for basket in baskets:
            if candidate_set.issubset(set(basket)):
                support[candidate] = support[candidate] + 1 if candidate in support else 1
    #print(support)
    return support

def prune_candidates(items, support):
    filtered_keys = list(dict(filter(lambda item : item[1] >= support, items.items())).keys())
    filtered_keys = sorted(filtered_keys)
    return filtered_keys

def son(partition, candidates):
    result = {}
    baskets = list(partition)
    result = get_support_candidates(candidates, baskets)
    return list(result.items())

def write_data_to_file(file_path, file_mode, item_sets, data_type):
    file=open(file_path,file_mode);
    file.write(data_type);
    file.write("\n")
    
    result = defaultdict(lambda : [])
    
    for item in item_sets:
        length = len(item)
        if(length == 1):
            result[length].append("(" + item[0] + ")")
        else:
            result[length].append(str(item))
    keys = sorted(list(result.keys()))
    
    for key in keys:
        string = ",".join(sorted(result[key]))
        file.write(str(string))
        file.write("\n\n")
    
    file.close()
        
            
    
    

In [12]:
initial_rdd = sc.textFile(input_file_path)
first = initial_rdd.first()
initial_rdd = initial_rdd.filter(lambda x : x != first).map(lambda line: line.split(","))
if case == 2:
    initial_rdd = initial_rdd.map(lambda x : [x[1], x[0]])

rdd = initial_rdd.groupByKey().map(lambda x : list(set(x[1])))
num_partitions = rdd.getNumPartitions()
total_length = rdd.count()
candidate_items = rdd.mapPartitions(lambda partition : apriori(partition, total_length)).flatMap(lambda result : result).distinct().collect()
write_data_to_file(output_file_path, "w", candidate_items, "Candidates:")
#print(candidate_items)

son = rdd.mapPartitions(lambda partition : son(partition, candidate_items)).reduceByKey(lambda a,b : a+b).filter(lambda frequent_item : frequent_item[1] >= threshold).map(lambda frequent_item : frequent_item[0]).collect()
#son = rdd.mapPartitions(lambda partition : son(partition, candidate_items)).reduceByKey(lambda a,b : a+b).collect()
write_data_to_file(output_file_path, "a", son, "Frequent Itemsets:")
#print(son)


/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
